In [1]:
import numpy as np 
import pandas as pd 
import json
from collections import OrderedDict

import os
print(os.path.abspath('.'))

/home/gdp/data/DeepPurpose0/VAE_data


In [2]:
#--------------------------------------------------------------------------

In [3]:
# KIBA
with open('target_seq.txt') as f:
    target = json.load(f)

with open('SMILES.txt') as f:
    drug = json.load(f)

target_kiba = list(target.values())
drug_kiba = list(drug.values())

In [4]:
len(target_kiba), len(drug_kiba)

(229, 2111)

In [5]:
# BindingDB
Smile = np.load("DB_smiles.npy", allow_pickle=True)
Target = np.load("DB_targets.npy", allow_pickle=True)

target_db = Target.tolist()
drug_db = Smile.tolist()

In [6]:
len(set(target_db)), len(set(drug_db))

(1413, 10661)

In [7]:
# DTC
data=pd.read_csv('data_DTC_addsmile_addtarget.csv')

drug_dtc = data['smile'].values.tolist()
target_dtc = data['target'].values.tolist()

In [8]:
len(set(target_dtc)), len(set(drug_dtc))

(895, 7626)

In [9]:
# Davis
ligands = json.load(open("ligands_can.txt"), object_pairs_hook=OrderedDict)
proteins = json.load(open("proteins.txt"), object_pairs_hook=OrderedDict)
affinity = pd.read_csv('affinity.txt', header=None, sep=' ')

drug_dav = []
prots_dav = []
y=[]
for d in ligands.keys():
    drug_dav.append(ligands[d])
for t in proteins.keys():
    prots_dav.append(proteins[t])

In [10]:
len(set(prots_dav)), len(set(drug_dav))

(379, 68)

In [11]:
#--------------------------------------------------------------------------
# 输出看一下具体情况

In [12]:
# davis
set_same = set(drug_dtc) & set(drug_dav)
list_same = list(set_same)
len(list_same)

0

In [13]:
set_same = set(target_dtc) & set(prots_dav)
list_same = list(set_same)
len(list_same)

335

In [14]:
# same drugs
#DB AND DTC有，而KIBA与这两个没相同drug
set_same = set(drug_db) & set(drug_dav)
list_same = list(set_same)
len(list_same)

0

In [15]:
# same targets
set_same = set(prots_dav) & set(target_db)
list_same = list(set_same)
len(list_same)

343

In [16]:
# same targets
set_same = set(target_kiba) & set(target_dtc)
list_same = list(set_same)
len(list_same)

198

In [17]:
# same targets
set_same = set(target_dtc) & set(target_db)
list_same = list(set_same)
len(list_same)

781

In [ ]:
#--------------------------------------------------------------------------

In [ ]:
# 1.unseen drugs, seen targets, 考虑如下三种情况：
# KIBA→DTC ,即KIBA中训练，在DTC子集上验证
# KIBA→DB ,即KIBA中训练，在DB子集上验证
# DTC→DB ,即DTC中训练，在DB子集上验证

In [19]:
# same targets
# 更改数据集依次得到三种情况下的same set,例子只给一个
set_same = set(target_dtc) & set(target_db)
list_same = list(set_same)
target_same = np.asarray(list_same)

In [20]:
set_same = set(drug_db) & set(drug_dtc)
list_same = list(set_same)
drug_same = np.asarray(list_same)

In [21]:
Smile = np.load("DB_smiles.npy", allow_pickle=True)
Target = np.load("DB_targets.npy", allow_pickle=True)
y = np.load("DB_y.npy", allow_pickle=True)
print('Done!')

df_DB = pd.DataFrame(zip(Smile, Target, y))
df_DB.rename(columns={0:'SMILES',1: 'Target_Sequence',2: 'Label'}, inplace=True)
print('in total: ' + str(len(df_DB)) + ' drug-target pairs')

Done!
in total: 66434 drug-target pairs


In [22]:
Smile = np.load("DTC_smiles_reg.npy", allow_pickle=True)
Target = np.load("DTC_targets_reg.npy", allow_pickle=True)
y = np.load("DTC_y_reg.npy", allow_pickle=True)
print('Done!')

df_DTC = pd.DataFrame(zip(Smile, Target, y))
df_DTC.rename(columns={0:'SMILES',1: 'Target_Sequence',2: 'Label'}, inplace=True)
print('in total: ' + str(len(df_DTC)) + ' drug-target pairs')

Done!
in total: 61479 drug-target pairs


In [13]:
affinity = pd.read_csv('affinity.txt', header=None, sep = '\t')
affinity = affinity.fillna(-1)
with open('target_seq.txt') as f:
    target = json.load(f)

with open('SMILES.txt') as f:
    drug = json.load(f)

target = list(target.values())
drug = list(drug.values())

SMILES = []
Target_seq = []
y = []
for i in range(len(drug)):
    for j in range(len(target)):
        if affinity.values[i, j] != -1:
            SMILES.append(drug[i])
            Target_seq.append(target[j])
            y.append(affinity.values[i, j])

Smile, Target, y = np.array(SMILES), np.array(Target_seq), np.array(y)

df_KIBA = pd.DataFrame(zip(Smile, Target, y))
df_KIBA.rename(columns={0:'SMILES',1: 'Target_Sequence',2: 'Label'}, inplace=True)
print('in total: ' + str(len(df_KIBA)) + ' drug-target pairs')

in total: 118254 drug-target pairs


In [23]:
# 根据same set 筛选出db的子集
df_DB_drugs_targets = df_DB[df_DB['Target_Sequence'].isin(target_same)]
df_DB_undrugs_targets = df_DB_drugs_targets[~df_DB_drugs_targets['SMILES'].isin(drug_same)]

In [24]:
len(df_DB_undrugs_targets)

50947

In [25]:
df_DB_undrugs_targets.to_csv('df_DB_undrugs_targets.csv', index = False)

In [26]:
df_DB_drugs_untargets = df_DB[~df_DB['Target_Sequence'].isin(target_same)]
df_DB_undrugs_untargets = df_DB_drugs_untargets[~df_DB_drugs_untargets['SMILES'].isin(drug_same)]

In [27]:
len(df_DB_undrugs_untargets)

7547

In [28]:
df_DB_undrugs_untargets.to_csv('df_DB_undrugs_untargets.csv', index = False)

In [16]:
# 根据same set 筛选出dtc的子集
df_DTC_drugs_untargets = df_DTC[~df_DTC['Target_Sequence'].isin(target_same)]
df_DTC_undrugs_untargets = df_DTC_drugs_untargets[~df_DTC_drugs_untargets['SMILES'].isin(drug_same)]

In [17]:
len(df_DTC_undrugs_untargets)

1387

In [18]:
df_DTC_undrugs_untargets.to_csv('df_DTC_undrugs_untargets.csv', index = False)

In [36]:
davis=pd.read_csv('df_davis.csv')

In [37]:
len(davis)

30056

In [38]:
df_davis_undrugs_targets = davis[davis['Target Sequence'].isin(target_same)]
len(df_davis_undrugs_targets)

25228

In [39]:
df_davis_undrugs_targets.to_csv('df_davis_undrugs_targets_dtcdavis.csv', index = False)

In [16]:
# same targets
set_same = set(target_dtc) & set(target_db)
list_same = list(set_same)
target_same = np.asarray(list_same)
# same targets
set_same = set(drug_db) & set(drug_dtc)
list_same = list(set_same)
drug_same = np.asarray(list_same)

In [18]:
# 根据same set 筛选出DTC的子集
df_DTC_drugs_targets = df_DTC[df_DTC['SMILES'].isin(drug_same)]
df_DTC_drugs_untargets = df_DTC_drugs_targets[~df_DTC_drugs_targets['Target_Sequence'].isin(target_same)]

In [22]:
len(df_DTC_drugs_untargets)

157

In [23]:
df_DTC_drugs_untargets.to_csv('df_DTC_drugs_untargets.csv', index = False)

In [19]:
DTC_undrugs_targets.to_csv('DTC_undrugs_targets.csv', index = False)
DTC_undrugs_untargets.to_csv('DTC_undrugs_untargets.csv', index = False)

In [ ]:
# 根据same set 筛选出DB的子集
DB_undrugs_targets = df_DB[df_DB['Target Sequence'].isin(target_same)]

# 根据same set 筛选出DB的子集，用于第四种情况；unseen drugs, unseen targets
DB_undrugs_untargets = df_DB[~df_DB['Target Sequence'].isin(target_same)]

In [ ]:
# 根据same set 筛选出DB的子集，注意DB与DTC有相同的drugs，需剔除！
DTC_drugs_targets = df_DTC[df_DTC['Target Sequence'].isin(target_same)]

set_same = set(drug_db) & set(drug_dtc)
list_same = list(set_same)
drug_same = np.asarray(list_same)
DTC_undrugs_targets = DTC_drugs_targets[~DTC_drugs_targets['SMILES'].isin(drug_same)]

# 根据same set 筛选出DTC的子集，用于第四种情况；unseen drugs, unseen targets
DTC_drugs_untargets = df_DTC[~df_DTC['Target Sequence'].isin(target_same)]
DTC_undrugs_untargets = DTC_drugs_untargets[~DTC_drugs_untargets['SMILES'].isin(drug_same)]

In [ ]:
#-----------------

In [ ]:
# 1.seen drugs, unseen targets, 考虑如下三种情况：
# DTC→DB ,即DTC中训练，在DB子集上验证
# DB→DTC ,即DB中训练，在DTC子集上验证
# KIBA→DAVIS ,即KIBA中训练，在DAVIS子集上验证,这里因为KIBA与DB和DTC没有共同drugs，所以考虑了DAVIS数据集

In [ ]:
# same targets
# 更改数据集依次得到三种情况下的same set
set_same = set(target_dtc) & set(target_db)
list_same = list(set_same)
target_same = np.asarray(list_same)

set_same = set(drug_dtc) & set(drug_db)
list_same = list(set_same)
drug_same = np.asarray(list_same)

In [ ]:
# 根据same set 筛选出DB的子集
DB_drugs_targets = df_DB[df_DB['SMILES'].isin(drug_same)]
DB_drugs_untargets = DB_drugs_targets[~DB_drugs_targets['Target Sequence'].isin(target_same)]

In [ ]:
# 根据same set 筛选出DTC的子集
DTC_drugs_targets = df_DTC[df_DTC['SMILES'].isin(drug_same)]
DTC_drugs_untargets = DTC_drugs_targets[~DTC_drugs_targets['Target Sequence'].isin(target_same)]

In [ ]:
# 根据same set 筛选出DAVIS的子集
DAV_drugs_targets = df_DAV[df_DAV['SMILES'].isin(drug_same)]
DAV_drugs_untargets = DAV_drugs_targets[~DAV_drugs_targets['Target Sequence'].isin(target_same)]